# Model Processing

## Imports & General Settings 

In [1]:
import unittest

from pycwt import Morlet
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm

# Our imports
from data import WaveletTransform, AFECGDataset, SecondDataset, WrapperDataset
import dsp
from model.blocks import ConvNet, BRNN, SoftmaxAttention
from model.baseline import Baseline
from training import train
import utils


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
test = unittest.TestCase()
plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## Dataset creation

In [4]:
dataset_name='afdb'
dataset = SecondDataset(dataset_name, '../data/files/', wt=WaveletTransform(Morlet(6), resample=20))
class_weights = dataset.load('./backup')

Loaded 2300 samples from backup
torch.Size([2300])
torch.Size([2300, 20, 375])


In [5]:
true_examples = dataset.samples[dataset.labels == 1]
false_examples = dataset.samples[dataset.labels == 0]
false_labels = dataset.labels[dataset.labels == 0]
augmented = []
augmented_labels = []
for i in range(len(false_examples) // len(true_examples)):
    augmented.append(true_examples.clone())
    augmented_labels.append(torch.ones(len(true_examples)).long())

print(torch.cat(augmented).shape)
print(false_examples.shape)
dataset.samples = torch.cat([torch.cat(augmented), false_examples])
dataset.labels = torch.cat([torch.cat(augmented_labels), false_labels])

torch.Size([2268, 20, 375])
torch.Size([2282, 20, 375])


In [6]:
print(dataset.labels.sum())
print(len(dataset) - dataset.labels.sum())

tensor(2268)
tensor(2282)


## Training

In [ ]:
heldout = int(len(dataset) * 0.2)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - heldout, heldout])

# model = Baseline(add_brnn=True, hidden_size=100)
model = nn.Sequential(
    ConvNet(size=(375, 20), batch=False),
    nn.Linear(50, 2)
)

config = dict(
    num_workers=8,
    batch_size=90,
    learning_rate=0.0005,
    weight_decay=0.01,
    class_weights=None,
    num_epochs=20,
    is_notebook=True
)

train(model, train_dataset, config)

## Testing

In [ ]:
y_true = dataset.labels[test_dataset.indices]
y_pred, test_acc = test(model, test_dataset, config)

In [ ]:
results = pd.DataFrame(classification_report(y_true, y_pred, zero_division=0, output_dict=True)).transpose()
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn+fp)
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
auc_score = metrics.auc(fpr, tpr)

In [ ]:
display(results)
print("Specificity:", specificity)
print("AUC:", auc_score)

In [ ]:
lr_auc = roc_auc_score(y_true, y_pred)
print('ROC AUC=%.3f' % (lr_auc))
lr_fpr, lr_tpr, _ = roc_curve(y_true, y_pred)
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Baseline model')

pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()
pyplot.show()

In [ ]:
pr_auc = average_precision_score(y_true, y_pred)
print("PR AUC:", specificity)